In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
path = '/content/drive/MyDrive/lecture/image_sensing/rockfall/keras-retinanet'

#作業ディレクトリをpathに移動する
os.chdir(path)

In [ ]:
!pip install -r requirements.txt

In [ ]:
cd keras-retinanet

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import tensorflow
from tensorflow import keras

import sys
sys.path.append('C:/Users/adachi/Desktop/adachi/stable_diffusion_esrgan_mubert/.venv/Lib/site-packages')
import cv2

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import time
import math

In [ ]:
# !pip install tensorflow==2.9.2
# !pip install matplotlib
# !pip install keras_tqdm

### Function for visualization

In [ ]:
#function for visualization
def vis_bb(model,df_label,path,img_path,score_threshold):
    image = read_image_bgr(path + img_path)

    # copy to draw on
    draw = image.copy()
    grays = cv2.cvtColor(draw, cv2.COLOR_RGB2GRAY)
    draw   = cv2.cvtColor(grays, cv2.COLOR_GRAY2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)
    labels = ['rockfall']
    labels_to_names = {0: 'rockfall'}

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < score_threshold:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption,1.5,2)

    #draw true bbox
    box_list = df_label[df_label['img_path'] == img_path].iloc[:,1:5].values
    for box in box_list:
        if math.isnan(box_list[0][0]) != True:
            b = box.astype(int)
            draw_box(draw, b, color=[255 , 0   , 0])
            caption = "Ground Truth"
            draw_caption(draw, b, caption,1.5,2)

    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(draw)
    plt.show()

### Training

In [ ]:
!python keras_retinanet/bin/train.py csv dataset/moon_mars/train_labels/train_labels_mix.csv dataset/moon_mars/train_labels/train_classes_mix.csv  

### Evaluation

In [ ]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = 'model_best/moon_mars/resnet50_csv_100.h5'

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.convert_model(model)

# load label to names mapping for visualization purposes
labels_to_names = {0: 'rockfall'}

df_label = pd.read_csv('dataset/moon_mars/test_labels/test_labels_mix.csv',names=('img_path', 'x1','y1','x2','y2','class_name'))

img_name_list = df_label['img_path'].unique()
for img_name in img_name_list:
  if math.isnan(df_label[df_label['img_path'] == img_name].iloc[:,1].values[0]) != True:
    df = df_label[df_label['img_path'] == img_name].iloc[:,1:5]
    df.insert(0, 'label', 'rockfall')
    name = img_name.replace('/','_')
    df.to_csv("pascalvoc_metrics/moon_mars/groundtruths/" + name+ ".txt", sep='\t', index=False,header=False)
    
process_time_list = []    
# load image
path = 'dataset/moon_mars/'

img_name_list = df_label['img_path'].unique()
for img_name in img_name_list:
  if math.isnan(df_label[df_label['img_path'] == img_name].iloc[:,1].values[0]) != True:

    image = read_image_bgr(path + img_name)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
   
    process_time = time.time() - start
    process_time_list.append(process_time)
    
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale
    df_pred = pd.DataFrame(boxes[0],columns = ('x1','y1','x2','y2'))
    df_pred.insert(0, 'label', 'rockfall')
    df_pred.insert(1, 'score', scores.T)
    df = df_pred[df_pred['score'] > 0]
    name = img_name.replace('/','_')
    df.to_csv("pascalvoc_metrics/moon_mars/detections/" + name+ ".txt", sep='\t', index=False,header=False)
    
process_time_array = np.array(process_time_list)
process_time_array = process_time_array[process_time_array < 1]
process_time_array.mean()    

threshold_list = [0.1,0.25,0.5,0.75]
for threshold in threshold_list:
    dir = 'pascalvoc_metrics/moon_mars/iou_'+str(threshold)
    if not(os.path.exists(dir)):
        os.mkdir(dir)
    !python pascalvoc_metrics/pascalvoc_metrics.py \
      --gt_dir pascalvoc_metrics/moon_mars/groundtruths \
      --gt_format xyrb \
      --det_dir pascalvoc_metrics/moon_mars/detections \
      --det_format xyrb \
      --iou $threshold \
      --output $dir

    df = pd.read_csv(dir + '/score_recall_precision.csv')
    ct_list = [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
    recall_list = []
    precision_list = []
    f1_list = []
    for ct in ct_list:    
        recall = df[df['Score'] > ct].iloc[0,1]
        precision = df[df['Score'] > ct].iloc[0,2]
        f1 = (2 * recall * precision) / (recall + precision)
        recall_list.append(recall)
        precision_list.append(precision)
        f1_list.append(f1)
    mylist = list(zip(ct_list,recall_list,precision_list,f1_list))
    df_metrics = pd.DataFrame(mylist,columns = ['ct','recall','precision','f1'])
    df_metrics['process_time'] = process_time_array.mean()
    df_metrics.to_csv(dir + '/result.csv',index = False)

### Measure the processing time

In [ ]:
import time
process_time_list = []
start = time.time()
# load image
path = 'dataset/moon_mars/'

img_name_list = df_label['img_path'].unique()
for img_name in img_name_list:
  if math.isnan(df_label[df_label['img_path'] == img_name].iloc[:,1].values[0]) != True:

    image = read_image_bgr(path + img_name)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    process_time = time.time() - start

    process_time_list.append(process_time)
process_time_array = np.array(process_time_list)
process_time_array.mean()

### Visualization

In [ ]:
img_name_list = df_label['img_path'].unique()
path = 'dataset/moon_mars/'
for img_path in img_name_list:
    vis_bb(model,df_label,path,img_path,0.2)